In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import Libraries

In [ ]:

from operator import index
from pyexpat.errors import XML_ERROR_TEXT_DECL
from turtle import home
from unicodedata import category
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import rand
import seaborn as sns

from sklearn.metrics import mean_squared_error

Seed = 42
np.random.seed(Seed)

### Build Funcitons


In [ ]:
# Build Funcitons

def save_csv(y_pred, file_name):
    submit = pd.read_csv(path + 'sample_submission.csv')
    submit['item_cnt_month'] = y_pred
    submit.to_csv(file_name, index = False)

### Load the datasets

In [ ]:
# Load the datasets
path = '/kaggle/input/competitive-data-science-predict-future-sales/'

train = pd.read_csv(path + './sales_train.csv')

shops = pd.read_csv(path + 'shops.csv')
items = pd.read_csv(path + 'items.csv')
item_categories = pd.read_csv(path + 'item_categories.csv')

#### merge categories_id in train set
- merge categories_id in train set
* drop string feature from dataset
* change negative values in item_cnt_day feature

In [ ]:
# merge categories_id in train set
train = train.merge(right=items, how='inner', on='item_id')

# drop string feature from dataset
train.drop(labels=['item_name'], axis = 1, inplace=True)

# change negative values in item_cnt_day feature
train['item_cnt_day'] = np.abs(train['item_cnt_day'])

grouped_train = train.groupby(by=['date_block_num', 'shop_id', 'item_category_id', 'item_id'], ).agg({'item_cnt_day':sum}).reset_index()


#### Shffle the dataframe first to random the rows
* split target from train 'item_cnt_day'
* drop item_cnt_day and date_block_num features

In [ ]:
# Shffle the dataframe first to random the rows
grouped_train = grouped_train.sample(frac=1, random_state=Seed)

# split target from train 'item_cnt_day'
target  = grouped_train['item_cnt_day']

# drop item_cnt_day and date_block_num features
grouped_train.drop(labels=['item_cnt_day', 'date_block_num'], axis =1, inplace=True)


#### show the correlation between features

In [ ]:
grouped_train.describe()

sns.heatmap(grouped_train.corr())


#### Split Train and valid data

In [ ]:
# Split Train and valid data
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(grouped_train, target, test_size=10_000)

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

#### Prepare test data

In [ ]:
# Prepare test data
test = pd.read_csv(path + 'test.csv')

test = test.merge(right=items, how='inner', on='item_id')
x_test = test.drop(labels=['item_name', 'ID'], axis=1)

# change the order of features ['shop_id', 'item_id', 'item_category_id'] to ['shop_id', 'item_category_id', 'item_id']
x_test = x_test[['shop_id', 'item_category_id', 'item_id']]


#### Scale the features

In [ ]:
# Scale the data using StandardScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(x_test)


#### Build the Models

##### 1 - RandomForest Algorithm

In [ ]:
''' 1- Random Forest Model'''
from sklearn.ensemble import RandomForestRegressor
rnd_reg = RandomForestRegressor(n_estimators=150, max_depth=20, random_state=Seed, 
    n_jobs=-1, criterion='mse')

print('Training the data....')
rnd_reg.fit(x_train, y_train)

# Score the model
print('Model Accuracy...')
score = rnd_reg.score(x_valid, y_valid)
print(score)

# Model RMSE
print('Model RMSE...')
y_pred_valid_rnd = rnd_reg.predict(x_valid)
rmse_rnd = mean_squared_error(y_valid, y_pred_valid_rnd, squared=False)
print(rmse_rnd)

# predict the test set
print('Predict the test data...')
y_pred_rnd = rnd_reg.predict(x_test)
y_pred_rnd = np.round(y_pred_rnd).astype(np.int64)
print(y_pred_rnd)

# Save the result
print('Save the prediction in submission file')
save_csv(y_pred_rnd, file_name = 'submission_solution_rnd_reg.csv')


##### 2- Suppor Vector Machine

In [ ]:
# ''' 2- Suppor Vector Machine '''

# from sklearn.svm import SVR
# svr_reg = SVR()

# print('Training the data....')
# svr_reg.fit(x_train, y_train)

# # Score the model
# print('Model Accuracy...')
# score = svr_reg.score(x_valid, y_valid)
# print(score)

# # Model RMSE
# print('Model RMSE...')
# y_pred_valid_svr = svr_reg.predict(x_valid)
# rmse_svr = mean_squared_error(y_valid, y_pred_valid_svr, squared=False)
# print(rmse_svr)

# # predict the test set
# print('Predict the test data...')
# y_pred_svr = svr_reg.predict(x_test)
# y_pred_svr = np.round(y_pred_svr).astype(np.int64)
# print(y_pred_svr)

# # Save the result
# print('Save the prediction in submission file')
# save_csv(y_pred_svr, file_name = 'submission_solution_svr_reg.csv')


##### 3- XGBRegressor

In [ ]:
'''3- XGBoost'''

import xgboost as xg
xg_reg = xg.XGBRegressor(n_estimators =120, learning_rate = .01, random_state = Seed, n_jobs = -1,)


print('Training the data....')
xg_reg.fit(x_train, y_train)

# Score the model
print('Model Accuracy...')
score = xg_reg.score(x_valid, y_valid)
print(score)

# Model RMSE
print('Model RMSE...')
y_pred_valid_xgb = xg_reg.predict(x_valid)
rmse_xgb = mean_squared_error(y_valid, y_pred_valid_xgb, squared=False)
print(rmse_xgb)

# predict the test set
print('Predicting the test data...')
y_pred_xgb = xg_reg.predict(x_test)
y_pred_xgb = np.round(y_pred_xgb).astype(np.int64)
print(y_pred_xgb)

# Save the result
print('Saving the prediction in submission file')
save_csv(y_pred_xgb, file_name = 'submission_solution_xg_reg.csv')